In [22]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from multiprocessing import cpu_count

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Subset, DataLoader

import skorch

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Dataset

In [57]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': cpu_count(), 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=64, shuffle=True, **kwargs)
test_loader = DataLoader(Subset(
    datasets.MNIST('/data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), indices=range(10000)),
    batch_size=1000, shuffle=True, **kwargs)

## Generic autoencoder class

In [52]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.writer = SummaryWriter(log_dir='/data/runs')
    
    def trains(self, device, train_loader, optimizer, epoch):
        self.train()
        loss_sum = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = self.compute_loss_train(data, target)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
            self.writer.add_scalar('Loss/train', loss.item(), epoch*len(train_loader)+batch_idx)
            
    def tests(self, device, test_loader):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                loss, output = self.compute_loss_test(data, target)
                test_loss += loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

## Autoencoder

In [64]:
class SimpleAutoencoder(Autoencoder):
    def __init__(self):
        super(SimpleAutoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output       
    
    def compute_loss_train(self, data, target):
        output = self(data)
        return F.nll_loss(output, target)
    
    def compute_loss_test(self, data, target):
        output = self(data)
        return F.nll_loss(output, target, reduction='sum').item(), output  # sum up batch loss

In [65]:
model = SimpleAutoencoder().to(device)
optimizer = optim.Adadelta(model.parameters())

# plot model
dataiter = iter(train_loader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# write to tensorboard
writer.add_image('mnist_images', img_grid)

scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.289280
Train Epoch: 1 [640/10000 (6%)]	Loss: 1.150923
Train Epoch: 1 [1280/10000 (13%)]	Loss: 0.593732
Train Epoch: 1 [1920/10000 (19%)]	Loss: 0.349042
Train Epoch: 1 [2560/10000 (25%)]	Loss: 0.475937
Train Epoch: 1 [3200/10000 (32%)]	Loss: 0.409502
Train Epoch: 1 [3840/10000 (38%)]	Loss: 0.223106
Train Epoch: 1 [4480/10000 (45%)]	Loss: 0.362938
Train Epoch: 1 [5120/10000 (51%)]	Loss: 0.328892
Train Epoch: 1 [5760/10000 (57%)]	Loss: 0.277428
Train Epoch: 1 [6400/10000 (64%)]	Loss: 0.252072
Train Epoch: 1 [7040/10000 (70%)]	Loss: 0.154685
Train Epoch: 1 [7680/10000 (76%)]	Loss: 0.189546
Train Epoch: 1 [8320/10000 (83%)]	Loss: 0.134101
Train Epoch: 1 [8960/10000 (89%)]	Loss: 0.217635
Train Epoch: 1 [9600/10000 (96%)]	Loss: 0.265757

Test set: Average loss: 0.1459, Accuracy: 9524/10000 (95%)

Train Epoch: 2 [0/10000 (0%)]	Loss: 0.179932
Train Epoch: 2 [640/10000 (6%)]	Loss: 0.118752
Train Epoch: 2 [1280/10000 (13%)]	Loss: 0.062095
Train Epoch: 2 [1920

KeyboardInterrupt: 

## Gaussian Variational Autoencoder

In [76]:
class VAE(Autoencoder):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(9216, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD
    
    def compute_loss_train(self, data, target):
        recon_batch, mu, logvar = self(data)
        return self.loss_function(recon_batch, data, mu, logvar)
    
    def compute_loss_test(self, data, target):
        recon_batch, mu, logvar = self(data)
        return loss_function(recon_batch, data, mu, logvar).item(), recon_batch  # sum up batch loss

In [77]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


scheduler = StepLR(optimizer, step_size=1)
for epoch in range(1, 14 + 1):
    model.trains(device, train_loader, optimizer, epoch)
    model.tests(device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/10000 (0%)]	Loss: 34984.750000
Train Epoch: 1 [640/10000 (6%)]	Loss: -119020.476562
Train Epoch: 1 [1280/10000 (13%)]	Loss: -312348.093750
Train Epoch: 1 [1920/10000 (19%)]	Loss: -404858.000000
Train Epoch: 1 [2560/10000 (25%)]	Loss: -461403.375000
Train Epoch: 1 [3200/10000 (32%)]	Loss: -522327.500000
Train Epoch: 1 [3840/10000 (38%)]	Loss: -512915.281250
Train Epoch: 1 [4480/10000 (45%)]	Loss: -582033.312500
Train Epoch: 1 [5120/10000 (51%)]	Loss: -608601.687500
Train Epoch: 1 [5760/10000 (57%)]	Loss: -673811.187500
Train Epoch: 1 [6400/10000 (64%)]	Loss: -716735.125000
Train Epoch: 1 [7040/10000 (70%)]	Loss: -753495.875000
Train Epoch: 1 [7680/10000 (76%)]	Loss: -725680.875000
Train Epoch: 1 [8320/10000 (83%)]	Loss: -784300.750000
Train Epoch: 1 [8960/10000 (89%)]	Loss: -869890.375000
Train Epoch: 1 [9600/10000 (96%)]	Loss: -801809.312500

Test set: Average loss: -13184.4413, Accuracy: 0/10000 (0%)

Train Epoch: 2 [0/10000 (0%)]	Loss: -793056.875000
Train Epoch: 2 

KeyboardInterrupt: 

## Stick-breaking Autoencoder